### Weekly Project 21
#### IMDB Top 250 Shows Scrapper

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### 1) Scrapping the data of all 250 Movies 

#### 1.1) Requesting the data from the website

In [2]:
link = 'https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250'
res = requests.get(link)
soup = BeautifulSoup(res.content, 'html.parser')

#### 1.2) Scrapping the data

In [3]:
c = 1
data = []

for i in (soup.find('tbody', class_ = 'lister-list').find_all('tr')):
    
    rank     = c
    title_id = i.find('td', class_ = 'titleColumn').find('a').get('href').split('/')[2].strip()
    title    = i.find('td', class_ = 'titleColumn').find('a').text.strip()
    aired    = i.find('td', class_ = 'titleColumn').find('span', class_='secondaryInfo').text.strip('()').strip()
    rating   = i.find('td', class_='ratingColumn imdbRating').text.strip()
    
    data.append([rank, title_id, title, aired, rating])
    c += 1

df = pd.DataFrame(data, columns = ['Rank', 'Title_ID', 'Title', 'Aired_In', 'Rating'])
df.head(10)

,Rank,Title_ID,Title,Aired_In,Rating
0,1,tt5491994,Planet Earth II,2016,9.4
1,2,tt0903747,Breaking Bad,2008,9.4
2,3,tt0795176,Planet Earth,2006,9.4
3,4,tt0185906,Band of Brothers,2001,9.4
4,5,tt7366338,Chernobyl,2019,9.3
5,6,tt0306414,The Wire,2002,9.3
6,7,tt6769208,Blue Planet II,2017,9.2
7,8,tt0417299,Avatar: The Last Airbender,2005,9.2
8,9,tt2395695,Cosmos: A Spacetime Odyssey,2014,9.2
9,10,tt0141842,The Sopranos,1999,9.2


#### 1.3) Saving the dataframe

In [4]:
df.to_csv('TV_shows.csv', index = False)

#### 2) Scrapping TV Show's data Individually 

In [5]:
from selenium import webdriver
from tqdm import tqdm
import time

driver = webdriver.Chrome()
driver.get('https://www.imdb.com')

#### 2.1) Srapping the Data

In [8]:
data = []
c = 1
for i in tqdm(df['Title_ID']):
    
    driver.get('https://www.imdb.com/title/' + i)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    rank   = c
    id_    = i
    
    try:
        title = (soup.find('div', class_ = 'sc-b5e8e7ce-1 kNhUtn').find('h1', class_ = 'sc-b73cd867-0 cEmnhL').text)
    except:
        title = np.nan
    
    try:
        aired = soup.find('div', class_='sc-b5e8e7ce-2 AIESV').find_all('a')[0].text.strip()
    except:
        aired = np.nan
    
    try:
        certificate = soup.find('div', class_='sc-b5e8e7ce-2 AIESV').find_all('a')[1].text.strip()
    except:
        certificate = np.nan
    
    try:
        duration = (soup.find('ul', class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-8c396aa2-0 bwvZbJ baseAlt').find_all('li')[3].text.strip())
    except:
        duration = np.nan
        
    try:
        rating = soup.find('div', class_ = 'sc-f6306ea-3 eGCTqT').find('span').text.strip()
    except:
        rating = np.nan
        
    try:
        res         = requests.get ('https://www.imdb.com/title/' + i + '/ratings/')
        no_of_votes = (BeautifulSoup(res.content, 'html.parser').find('div', class_ = 'allText').text.strip().split('\n')[0])
    except:
        no_of_votes = np.nan
    
    try:
        stars = ','.join([cast.text.strip() for cast in soup.find('div', class_ ='ipc-metadata-list-item__content-container').find_all('a')])
    except:
        stars = np.nan
    
    try:
        genre = soup.find_all('div', class_ = 'ipc-metadata-list-item__content-container')[6].text.strip()
    except:
        genre = np.nan
        
    try:
        episodes = soup.find('span', class_ ='ipc-title__subtext').text.strip()
    except:
        episodes = np.nan
    
    try:
        seasons = soup.find_all('a', class_ = 'ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--on-accent2 ipc-text-button')[1].text.split(' ')[0].strip()
    except:
        seasons = np.nan
    
    data.append([rank, id_, title, aired, certificate, duration, rating, no_of_votes, stars, genre, episodes, seasons])
    c += 1

100%|██████████| 250/250 [26:43<00:00,  6.41s/it]


In [9]:
df_1 = pd.DataFrame(data, columns = ['Rank', 'Title_ID', 'Title', 'Aired_In', 'Certificate', 'Duration', 'Rating', 'Votes', 'Stars', 'Genre', 'Episodes', 'Seasons'])

In [15]:
df_1.head(10)

,Rank,Title_ID,Title,Aired_In,Certificate,Duration,Rating,Votes,Stars,Genre,Episodes,Seasons
240,241,tt11198330,House of the Dragon,2022–,TV-MA,NaN,8.5,"304,025","Ryan J. Condal,George R.R. Martin",5 wins & 20 nominations,11,1
241,242,tt0421291,Avrupa Yakasi,2004–2009,NaN,NaN,8.6,"19,402","Gülse Birsel,Levent Üzümcü,Senay Gürler",Sacit and Volkan never appeared together throu...,190,NaN
242,243,tt3920596,Big Little Lies,2017–2019,TV-MA,1h,8.5,"203,561",David E. Kelley,54 wins & 101 nominations total,14,NaN
243,244,tt1299368,Southland,2009–2013,TV-MA,42m,8.5,"27,331",Ann Biderman,15 wins & 26 nominations total,43,NaN
244,245,tt2937900,The Knick,2014–2015,TV-MA,58m,8.5,"49,605","Jack Amiel,Michael Begler",8 wins & 31 nominations total,21,NaN
245,246,tt0988818,Gintama,2005–2021,TV-14,25m,8.7,"12,140","Tomokazu Sugita,Daisuke Sakaguchi,Rie Kugimiya",AnimationActionComedySci-Fi,375,NaN
246,247,tt1877368,NaN,2010–,TV-PG,1h,8.6,"11,034","Paul Hollywood,Mary Berry,Prue Leith",TV-PG,165,NaN
247,248,tt0262150,Black Books,2000–2004,TV-PG,25m,8.5,"59,646",Dylan Moran,3 wins & 3 nominations total,18,NaN
248,249,tt0310455,Foyle's War,2002–2015,TV-14,1h 30m,8.6,"16,117",Anthony Horowitz,1 win & 4 nominations total,28,NaN
249,250,tt0472027,John Adams,2008,TV-14,1h 11m,8.5,"33,968","Paul Giamatti,Laura Linney,John Dossett",BiographyDramaHistory,7,1


#### 2.2) Saving the dataframe

In [16]:
df_1.to_csv("TV_shows(Individual).csv", index = False)